In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
x = tf.placeholder('float', [None, 784])  # x 可理解为一个图像数据集
W = tf.Variable(tf.zeros([784, 10]))  # W是全为0的二阶张量
b = tf.Variable(tf.zeros([10])) # b是全为0的一阶张量y 

y = tf.nn.softmax(tf.matmul(x, W) + b)
y_  = tf.placeholder('float', [None, 10]) #  y_ 表示yhat  y的预测值




In [10]:
# CNN 权重初始化应加入少量噪声阿里打破对称性以及避免0梯度
# 激活函数使用 Relu  
def weight_variable(shape):
    initial  =  tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1 , shape=shape)
    return tf.Variable(initial)

In [11]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') # 有0填充 保证卷积前后size不变

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')



In [12]:
# 第一层卷积

W_conv1 = weight_variable([5,5,1,32])  # 卷积核5*5大小   1个输入通道  算出32个特征
b_conv1 = bias_variable([32])

x_imge = tf.reshape(x, [-1, 28, 28, 1]) #  [  ,width, height, chanels]

h_conv1 = tf.nn.relu(conv2d(x_imge, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [13]:
#  第二层卷积 
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


In [14]:
#  全连接层 

W_fc1 = weight_variable([7 * 7 * 64 , 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 =  tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [15]:
#  为了减少过拟合，在输出层前加入dropout

keep_prob =  tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [16]:
#  输出层

W_fc2 =  weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv =  tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


In [20]:
#   训练和评估模型

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction =  tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
print ('correct_prediction' , type(correct_prediction))
print (correct_prediction.get_shape())

print ("~" * 50)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
print ('accuracy' , type(accuracy))


# 再用一个Session启动模型，并且，初始化变量
sess =  tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0 :
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print ("step %d, training accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    
    
print ("test accuracy %g"%accuracy.eval(session=sess, feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

correct_prediction <class 'tensorflow.python.framework.ops.Tensor'>
(?,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
accuracy <class 'tensorflow.python.framework.ops.Tensor'>


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`